# Boosting

We will look here into the practicalities of boosted trees. 

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn.apionly as sns
sns.set_style("whitegrid")
sns.set_context("poster")

## Dataset

First, the data. We will be attempting to predict the presidential election results (at the county level) from 2016, measured as 'votergap' = (trump - clinton) in percentage points, based mostly on demographic features of those counties.  Let's quick take a peak at the data:

In [2]:
elect_df = pd.read_csv("data/county_level_election.csv")
elect_df.head()

In [3]:
from sklearn.model_selection import train_test_split

In [11]:
# split 80/20 train-test
X = elect_df[['population','hispanic','minority','female','unemployed','income','nodegree','bachelor','inactivity','obesity','density','cancer']]
response = elect_df['votergap']
Xtraindf, Xtestdf, ytrain, ytest = train_test_split(X,response,test_size=0.2, random_state=1983)


In [12]:
plt.hist(ytrain)
Xtraindf.hist(column=['minority', 'population','hispanic','female']);

How would you describe these variables?

In [13]:
x = Xtraindf['minority'].values
o = np.argsort(x)
x = x[o]
y = ytrain.values
y = y[o]
plt.plot(np.log(x),y,'.')
xx = np.log(x).reshape(-1,1)

## Boosting Regression Trees

 Gradient Bossting is very state of the art, and has major connections to logistic regression, gradient descent in a functional space, and search in information space. See Schapire and Freund's MIT Press book for details (Google is a wonderful thing).

But briefly, let us cover the idea here. The idea is that we will use a bunch of weak 'learners' (aka, models) which are fit sequentially. The first one fits the signal, the second one the first model's residual, the third the second model's residual, and so on. At each stage we upweight the places that our previous model did badly on. First let us illustrate.

In [17]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor


In [20]:
# code from http://nbviewer.jupyter.org/github/pprett/pydata-gbrt-tutorial/blob/master/gbrt-tutorial.ipynb
import time
from IPython import display
plt.figure(figsize=(12, 8))
plt.plot(xx, y, '.');
counter = 0
estab = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=1),
                          n_estimators=500, learning_rate=1)
estab.fit(xx, y)
staged_predict_generator = estab.staged_predict(xx)
for stagepred in staged_predict_generator:
    counter = counter + 1
    if counter in [1, 2, 4, 8, 10, 50, 100, 200]:
        plt.plot(xx, stagepred, alpha=0.7, label=str(counter), lw=4)
        plt.legend();
        display.display(plt.gcf())
        display.clear_output(wait=True)
        time.sleep(2)

## Now on the whole data set

In [22]:
estab2 = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=2),
                          n_estimators=1000, learning_rate=0.3)
estab2.fit(Xtraindf, ytrain)
print(estab2.score(Xtraindf, ytrain))
print(estab2.score(Xtestdf, ytest))

staged_scores = estab2.staged_score(Xtest, ytest)
i=1
scores=[]
for stagesc in staged_scores:
    scores=np.append(scores,stagesc)
    i = i+1
plt.plot(scores);

Ok, so this demonstration helps us understand some things about boosting.

- `n_estimators` is the number of trees, and thus the stage in the fitting. It also controls the complexity for us. The more trees we have the more we fit to the tiny details.
- `staged_predict` gives us the prediction at each step
- once again `max_depth` from the underlying decision tree tells us the depth of the tree. But here it tells us the amount of features interactions we have, not just the scale of our fit. But clearly it increases the variance again.

Ideas from decision trees remain. For example, increase `min_samples_leaf` or decrease `max_depth` to reduce variance and increase the bias.

EXERCISE

1. What do you expect to happen if you increase `max_depth` to 5?  Edit the code above to explore the result.
2. What do you expect to happen if you put `max_depth` back to 1 and decrease the `learning_rate` to 0.1?  Edit the code above to explore the result.
3. Do a little work to find some sort of 'best' values of `max_depth` and `learning_rate`.  Does this result make sense?

**Answers**
1. Increasing `max_depth` for each model does not really improve things.  It will make each model more complex, which may not be a good thing.  We want lots of little models that each are not overfit, so allowing any single model to have a lot of complexity (and thus overfit) could be a problem.
2. The model will progress much more slowly to fitting to the data well.  This may be a good thing: this is essentially a form of 'regularization' in each model and will help prevent overfitting to any one model.  It allows future models to have more 'error' to be able to explain.
3. See code below.


In [25]:
from collections import OrderedDict
from sklearn.model_selection import GridSearchCV

param_dict3 = OrderedDict(
    n_estimators = [200],
    learning_rate = [0.2, 0.4, 0.6, 0.8]
)

# your code here


In [164]:
best_scores

### Deviance

A deviance plot can be used to compare train and test errors against the number of iterations.

Training error (deviance, related to the KL-divergence) is stored in est.train_score_
Test error is computed using est.staged_predict (this uses est.loss_)

In [33]:
from sklearn.ensemble import GradientBoostingRegressor

In [38]:
def deviance_plot(est, X_test, y_test, ax=None, label='', train_color='#2c7bb6', 
                  test_color='#d7191c', alpha=1.0, ylim=(-80, 100)):
    """Deviance plot for ``est``, use ``X_test`` and ``y_test`` for test error. """
    n_estimators = len(est.estimators_)
    test_dev = np.empty(n_estimators)

    for i, pred in enumerate(est.staged_predict(X_test)):
       test_dev[i] = est.loss_(y_test, pred)

    if ax is None:
        fig = plt.figure(figsize = (12, 8))
        ax = plt.gca()
        
    ax.plot(np.arange(n_estimators) + 1, test_dev, color=test_color, label='Test %s' % label, 
             linewidth=2, alpha=alpha)
    ax.plot(np.arange(n_estimators) + 1, est.train_score_, color=train_color, 
             label='Train %s' % label, linewidth=2, alpha=alpha)
    ax.set_ylabel('Error')
    ax.set_xlabel('n_estimators')
    #ax.set_ylim(ylim)
    return test_dev, ax

In [39]:
gb = GradientBoostingRegressor(n_estimators=125, max_depth=5)
gb.fit(Xtraindf, ytrain)
deviance_plot(gb, Xtest, ytest, ylim=(0,0.8));
plt.legend();

Notice the wide gap. This is an indication of overfitting!

Unlike random forests, where we are using the randomness to our benefits, the GBRT requires careful cross-validation

Peter Prettenhofer, who wrote sklearns GBRT implementation writes in his pydata14 talk (worth watching!)

Hyperparameter tuning I usually follow this recipe to tune the hyperparameters:

- Pick n_estimators as large as (computationally) possible (e.g. 3000)
- Tune max_depth, learning_rate, min_samples_leaf, and max_features via grid search
- A lower learning_rate requires a higher number of n_estimators. Thus increase n_estimators even more and tune learning_rate again holding the other parameters fixed
- This last point is a tradeof between number of iterations or runtime against accuracy. And keep in mind that it might lead to overfitting.

Let me add however, that poor learners do rather well. So you might want to not cross-validate max_depth. And min_samples_per_leaf is not independent either, so if you do use cross-val, you might just use one of those.

EXERCISE: use a param grid and `GridSearchCV` to try and overfit this model as little as possible. We'll only explore part of the space as this takes time.

eg:

```
param_grid = {'learning_rate': [0.1, 0.01],
              'min_samples_leaf': [3, 5],  ## depends on the num of training examples, or use max_depth
              'max_features': [0.2, 0.6]
              }
```

And remember you;ll have to figure the number of trees

In [ ]:
# your code here

Once you find the best model, make a deviance plot with the best params.

In [ ]:
# your code here

Also find the most informative features

### Note: what's the relationship between residuals and the gradient?

Pavlos showed in class that for the squared loss, taking the gradient in the "data point functional space", ie a N-d space for N data points with each variable being $f(x_i)$ just gives us the residuals. It turns out that the gradient descent is a more general idea, and one can use this for different losses. And the upweighting of poorly fit points in AdaBoost is simply a weighing by gradient. If the gradient (or residual) is high it means you are far away from optimum in this functional space, and if you are at 0, you have a flat gradient!

The ideas from the general theory of gradient descent tell us this: we can slow the learning by shrinking the predictions of each tree by some small number, which is called the learning_rate (learning_rate). This "shrinkage" helps us not overshoot, but for a finite number of iterations also simultaneously ensures we dont overfit by being in the neighboorhood of the minimum rather than just at it! But we might need to increase the iterations some to get into the minimum area.

---